In [14]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns

#Modeling 
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor
from sklearn.svm import SVC
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer

from catboost import CatBoostRegressor
from xgboost import XGBRegressor

import warnings
warnings.filterwarnings('ignore')



In [6]:
df = pd.read_csv('data/stud.csv')
df.head()

,gender,race_ethnicity,parental_level_of_education,lunch,test_preparation_course,math_score,reading_score,writing_score
0,female,group B,bachelor's degree,standard,none,72,72,74
1,female,group C,some college,standard,completed,69,90,88
2,female,group B,master's degree,standard,none,90,95,93
3,male,group A,associate's degree,free/reduced,none,47,57,44
4,male,group C,some college,standard,none,76,78,75


Preparing X and y variables

In [7]:
X = df.drop('math_score', axis=1)
y = df['math_score']

In [9]:
num_features = X.select_dtypes(exclude='object').columns
cat_features = X.select_dtypes(include='object').columns

num_transformer = StandardScaler()
oh = OneHotEncoder()

In [10]:
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), num_features),
        ('cat', OneHotEncoder(), cat_features)
    ]
)

In [11]:
X = preprocessor.fit_transform(X)

In [17]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train.shape, X_test.shape

((800, 19), (200, 19))

Create an Evaluate Function to give all metrics after model Training 

In [19]:
def evaluate_model(true, predicted):
    mae = mean_absolute_error(true, predicted)
    mse = mean_squared_error(true, predicted)
    rmse = np.sqrt(mean_squared_error(true, predicted))
    r2 = r2_score(true, predicted)
    return mae, mse, rmse, r2

In [20]:
models = {
    'LinearRegression' : LinearRegression(), 
    'Lasso' : Lasso(), 
    'Ridge' : Ridge(), 
    'KNeighborsRegressor' : KNeighborsRegressor(), 
    'DecisionTreeRegressor' : DecisionTreeRegressor(),
    'RandomForestRegressor' : RandomForestRegressor(), 
    'XGBRegressor' : XGBRegressor(), 
    'CatBoostRegressor' : CatBoostRegressor(verbose=False),
    'AdaBoostRegressor' : AdaBoostRegressor()
}

model_list = []
r2_list = []

for key in models:
    model = models[key]
    model.fit(X_train, y_train)

    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)

    train_mae, train_mse, train_rmse, train_r2 = evaluate_model(y_train, y_train_pred)
    test_mae, test_mse, test_rmse, test_r2 = evaluate_model(y_test, y_test_pred)

    print(key)
    model_list.append(key)

    print('Model performance for training set')
    print(f'- RMSE: {train_rmse:.4f}')
    print(f'- MAE: {train_mae:.4f}')
    print(f'- MSE: {train_mse:.4f}')
    print(f'- R2: {train_r2:.4f}')
    print('----------------------------')

    print('Model performance for test set')
    print(f'- RMSE: {test_rmse:.4f}')
    print(f'- MAE: {test_mae:.4f}')
    print(f'- MSE: {test_mse:.4f}')
    print(f'- R2: {test_r2:.4f}')

    r2_list.append(test_r2)

    print('='*35)
    print('\n')


LinearRegression
Model performance for training set
- RMSE: 5.3231
- MAE: 4.2667
- MSE: 28.3349
- R2: 0.8743
----------------------------
Model performance for test set
- RMSE: 5.3940
- MAE: 4.2148
- MSE: 29.0952
- R2: 0.8804


Lasso
Model performance for training set
- RMSE: 6.5938
- MAE: 5.2063
- MSE: 43.4783
- R2: 0.8071
----------------------------
Model performance for test set
- RMSE: 6.5197
- MAE: 5.1579
- MSE: 42.5063
- R2: 0.8253


Ridge
Model performance for training set
- RMSE: 5.3233
- MAE: 4.2650
- MSE: 28.3378
- R2: 0.8743
----------------------------
Model performance for test set
- RMSE: 5.3904
- MAE: 4.2111
- MSE: 29.0563
- R2: 0.8806


KNeighborsRegressor
Model performance for training set
- RMSE: 5.7128
- MAE: 4.5168
- MSE: 32.6355
- R2: 0.8552
----------------------------
Model performance for test set
- RMSE: 7.2538
- MAE: 5.6190
- MSE: 52.6170
- R2: 0.7838


DecisionTreeRegressor
Model performance for training set
- RMSE: 0.2795
- MAE: 0.0187
- MSE: 0.0781
- R2: 0

In [21]:
pd.DataFrame(list(zip(model_list, r2_list)), columns=['Model', 'R2_score']).sort_values(ascending=False, by=['R2_score'])

,Model,R2_score
2,Ridge,0.880593
0,LinearRegression,0.880433
5,RandomForestRegressor,0.851650
7,CatBoostRegressor,0.851632
8,AdaBoostRegressor,0.843439
1,Lasso,0.825320
6,XGBRegressor,0.821221
3,KNeighborsRegressor,0.783770
4,DecisionTreeRegressor,0.737136


In [ ]:
df.head()